<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# LinkedIn - Send interactions analysis by email with chat plugin CTA

**Tags:** #linkedin #post #interactions #naas_drivers #content #snippet #dataframe #growth

**Author:** [Jeremy Ravenel](https://www.linkedin.com/in/jeremyravenel/)

**Description:** This notebook gets the list of LinkedIn profile that interacted with a post, determine if they fit the ICP using prompt engineering, and sends a report via email with a CTA that opens a Chat Agent to explore outreach possibilities.


<div class="alert alert-info" role="info" style="margin: 10px">
<b>Disclaimer:</b><br>
This code is in no way affiliated with, authorized, maintained, sponsored or endorsed by Linkedin or any of its affiliates or subsidiaries. It uses an independent and unofficial API. Use at your own risk.

This project violates Linkedin's User Agreement Section 8.2, and because of this, Linkedin may (and will) temporarily or permanently ban your account. We are not responsible for your account being banned.
<br>
</div>

## Input

### Import libraries

In [ ]:
import os
import json
import pandas as pd

import naas 
import naas_drivers
import openai

from naas_drivers import emailbuilder, linkedin
from datetime import datetime, date
import random
import time
from dateutil.parser import parse
import matplotlib.pyplot as plt
import plotly.graph_objects as go

import plotly.io as pio
try:
    import tiktoken
except:
    !pip install tiktoken --user
    import tiktoken

### Setup variables
<a href='https://www.notion.so/LinkedIn-driver-Get-your-cookies-d20a8e7e508e42af8a5b52e33f3dba75'>How to get your cookies ?</a>

In [ ]:
# Define scenario 
is_demo = False # Default to True

# For LinkedIn 
POST_URL = "https://www.linkedin.com/posts/jeremyravenel_ai-linkedin-chatgpt-activity-7089300798962028544-viOU/"
refresh_interval = 2 #time in minutes between two LinkedIn queries
LI_AT = naas.secret.get("LINKEDIN_LI_AT")  # EXAMPLE : "AQFAzQN_PLPR4wAAAXc-FCKmgiMit5FLdY1af3-2"
JSESSIONID = naas.secret.get("LINKEDIN_JSESSIONID")   # EXAMPLE : "ajax:8379907400220387585"
PROFILE_URL = naas.secret.get("LINKEDIN_PROFILE_URL")  # EXAMPLE "https://www.linkedin.com/in/myprofile/"

# For Emails
EMAIL_TO = "jeremy@naas.ai"  # you will receive weekly summary at this email
EMAIL_FROM = None  # summary will have this email as sender. Only available for your naas email, otherwise you will receive this email from notification@naas.ai
EMAIL_SUBJECT = (f"🚀 Growth Engine Demo - Email Update, {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
DATE_FORMAT = "%Y-%m-%d"


# For openAI API
openai_api_key = naas.secret.get("OPENAI_API_KEY")

# For AI Chat Plugin
plugin_name = "🚀 Growth Agent Demo"
plugin_model = "gpt-3.5-turbo-16k"
plugin_temperature = 0
plugin_max_tokens = 8192
system_prompt_max_tokens = 2084

# For Asset Generation
output_dir = "../outputs/"
csv_file_name = "linkedin_interactions.csv"
image_file_name = "growth_analysis.png"
plugin_file_name = "growth_plugin.json"

## Model

### Setup directories

In [ ]:
# Check if directory exists and create it if not
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
# Generate outputs files path
csv_file_path = os.path.join(output_dir, csv_file_name)
image_file_path = os.path.join(output_dir, image_file_name)
plugin_file_path = os.path.join(output_dir, plugin_file_name)
print('📂 CSV file path:', csv_file_path)
print('📂 Image file path:', image_file_path)
print('📂 Plugin file path:', plugin_file_path)

### Get or load data

**Available columns :**
- PROFILE_ID : LinkedIn unique profile ID
- PROFILE_URL : LinkedIn unique profile URL
- PUBLIC_ID : LinkedIn public profile ID
- FIRSTNAME : First name of profile
- LASTNAME : Last name of profile
- FULLNAME : First name + Last name
- OCCUPATION : Headline in profile description
- PROFILE_PICTURE : Profile picture URL
- BACKGROUND_PICTURE : Background picture URL
- REACTION_TYPE : Post reaction type
- POST_URL : Post URL

In [ ]:
def get_or_load_data(
    li_at,
    JSESSIONID,
    POST_URL,
    file_path,
    refresh_interval,
    demo_mode=is_demo  
):
    # Init
    df = pd.DataFrame()
    update_data = True
    
    # Check if demo_mode is True
    if demo_mode:
        file_path = "../inputs/demo_data-linkedin_interactions.csv"
        if os.path.exists(file_path):
            # Read file
            df = pd.read_csv(file_path)
            df.to_csv(csv_file_path, index=False)
            return df
    
    # Check if output already exists
    if os.path.exists(csv_file_path):
        # Read file
        df = pd.read_csv(csv_file_path)
        
        # Assess if LinkedIn API can be invoked based on the last call.
        # To emulate human interaction, we must avoid making excessive calls to the LinkedIn API. Overdoing this could result in being banned.
        if len(df) > 0:
            if "DATE_EXTRACT" in df.columns:
                # Manage calls to API
                last_update_date = df.loc[0, "DATE_EXTRACT"]
                time_last_update = datetime.now() - datetime.strptime(last_update_date, "%Y-%m-%d %H:%M:%S")
                minute_last_update = time_last_update.total_seconds() / 60
                if minute_last_update < refresh_interval:
                    update_data = False
                    print(f"ℹ️ Nothing to update as the last retrieve was {int(minute_last_update)} minutes ago.")
                
    if update_data:
        # Get last posts
        df = linkedin.connect(LI_AT, JSESSIONID).post.get_likes(POST_URL)
        # Save last posts in CSV
        df.to_csv(csv_file_path, index=False)
        print("💾 Dataframe successfully saved:", csv_file_path)
    return df

# Call the function and display results
df = get_or_load_data(LI_AT, JSESSIONID, POST_URL, csv_file_path, refresh_interval)
df

### Prepare data for prediction

In [ ]:
df['summary'] = df['OCCUPATION']

### Determine if profiles match with ICP

In [ ]:
def predict_category(summary):
    openai.api_key = naas.secret.get("OPENAI_API_KEY")

    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=f"""I am building Naas, the Universal open source data plaform.
        I have 2 ideal customer profile, one is a 'data producer' with basic knowledge of Python that could use our Notebook templates to create plugins. 
        These plugions are then distributed data via our NaasAI Chat interface. 
        The other one is a 'data consummer' that will enjoy using NaasAI Chat for its basic LLMs integration but also interested in having its own data available, hence work with the data producer. 
        I will give you the {summary} from a profile I get from LinkedIn, you will return stricly and only one of the following values inside the simple quotes based on the best match 'DataProducer', 'DataConsummer', 'NotICP' or 'Don't know' if you don't find a plausible match with the first 3 values.
        Don't put the results into quotes.
        """,
        temperature=0,
        max_tokens=60
    )

    return response.choices[0].text.strip()

# Apply the function to the 'summary' column
df['ICP'] = df['summary'].apply(predict_category)

In [ ]:
def group_and_count(df):
    return df.groupby('ICP').size().reset_index(name='SUM')

df_icp = group_and_count(df)
df_icp

### Create chart

In [ ]:
def create_vertical_barchart(df_icp,
                               label="ICP",
                               value="SUM",
                               value_d="SUM"):
    # Init
    fig = go.Figure()
    
    # Return empty fig if dataframe is empty
    if len(df_icp) == 0:
        return fig
    
    # Create fig
    fig.add_trace(
        go.Bar(
            y=df_icp[value],
            x=df_icp[label],
            textposition="outside",
            marker=dict(color="#0f93d2"),
            orientation="v"
        )
    )
    
    # Add logo
    fig.add_layout_image(
        dict(
            #source="logo.png",
            xref="paper",
            yref="paper",
            x=0.28,
            y=.035,
            sizex=0.15,
            sizey=0.15,
            xanchor="right",
            yanchor="bottom"
        )
    )
    
    fig.update_traces(showlegend=False)
    
    # Plotly: Create title
    total_value = "{:,.0f}".format(df_icp[value].sum()).replace(",", " ")
    title = f"<b><span style='font-size: 20px;'>ICP Matches from last post</span></b>"
    fig.update_layout(
        title=title,
        #title_x=0.09,
        title_font=dict(family="Arial", color="black"),
        paper_bgcolor="#ffffff",
        plot_bgcolor="#ffffff",
        width=1200,
        height=600,
        #margin_pad=10,
        #margin_r=10,
        #margin_l=10,
    )
    
    fig.update_xaxes(showticklabels=True)
    
    return fig

create_vertical_barchart(df_icp)

### Create asset from chart

In [ ]:
# Create the chart
chart = create_vertical_barchart(df_icp)

# Save as PNG
pio.write_image(chart, image_file_path)

#graph_url = naas.asset.add("chart.html", {"inline": True})
graph_image = naas.asset.add(image_file_path)

### Set email parameters

In [ ]:
today = datetime.now().strftime(DATE_FORMAT)
today

### Prepare data for email

In [ ]:
# Transpose the DataFrame
df_icp_transposed = df_icp.transpose()

# Set the column names to the first row's values
df_icp_transposed.columns = df_icp_transposed.iloc[0]

# Drop the first row
df_icp_transposed = df_icp_transposed.drop(df_icp_transposed.index[0])

# Capitalize, remove spaces, and replace empty strings with 'unknown'
df_icp_transposed.columns = [str(col).replace(' ', '').capitalize() or 'unknown' for col in df_icp_transposed.columns]

# Remove leading and trailing whitespaces
df_icp_transposed.columns = [str(col).strip().capitalize() or 'unknown' for col in df_icp_transposed.columns]
df_icp_transposed

In [ ]:
Dataconsummer = df_icp_transposed.Dataconsummer.sum()
Dataproducer= df_icp_transposed.Dataproducer.sum()
Noticp = df_icp_transposed.Noticp.sum()

# preview of what will be send by email:
print(
    "\n\t👩‍💼 Data consummer\t",
    Dataconsummer,
    "\n\t👷 Data producer\t",
    Dataproducer,
    "\n\t❌ Not ICP\t",
    Noticp,
)

### Create Naas Chat plugin

In [ ]:
system_prompt = f"""Act as a Growth Agent who has access to a list of interactions from content that enable the user to get qualified contacts.
Your role is to manage and optimize the list of people who interacted on the content, ensuring to extract only the most qualified contacts to feed the sales representative.
You will need to analyze data to define preferences on how to touch the contact needs and interests.
Your ultimate goal is to create engaging and relevant list of qualified leads, contributing to the overall success of the sales & marketing strategy.
The first message should be about presenting yourself with a maximum of 5 bullet points and displaying the current content analytics data to be displayed as an image inside the markdown of the chat:{graph_image}.
Then, wait for the first answer from the user, and then start with the first high-level analysis.
Here is the data from the content analytics that you should focus on: {df}
"""

### Check token count 

In [ ]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

system_prompt_tokens = num_tokens_from_string(system_prompt, "cl100k_base")
if system_prompt_tokens > system_prompt_max_tokens:
    print("⚠️ Be carefull, your system prompt looks too big. Tokens:", system_prompt_tokens)
else:
    print("✅ System prompt tokens count OK:", system_prompt_tokens)

### Generate plugin

In [ ]:
# Create json
plugin = {
    "name": plugin_name,
    "model": plugin_model,
    "temperature": plugin_temperature,
    "max_tokens": plugin_max_tokens,
    "prompt": system_prompt,
}

# Save dict to JSON file
with open(plugin_file_path, "w") as f:
    json.dump(plugin, f)
print("💾 Plugin successfully saved:")

plugin = naas.asset.add(plugin_file_path, params={"inline": True})

### Generate email content

In [ ]:
def format_number(num):
    NUMBER_FORMAT = "{:,.0f}"
    num = str(NUMBER_FORMAT.format(num)).replace(",", " ")
    return num

In [ ]:
def email_brief(
    today,
    Dataconsummer,
    Dataproducer,
    Noticp,

):
    content = {
        'title': ("🚀 Growth Engine - Email Update"),
        'heading': (f"Date:{today}"),
        "txt_intro": (
            f"Hi there,<br><br>" f"Here is your growth engine email as of {today}."
        ),
        "title_1": emailbuilder.text(
            "Overview", font_size="27px", text_align="center", bold=True
        ),
        "text_1": emailbuilder.text(
            f"Your last content interactions has been categorized has follows:"
        ),
        "image_1": emailbuilder.image(graph_image),
        "list_1": emailbuilder.list(
            [
                f"👩‍💼 Data consummer : <b>{format_number(Dataconsummer)}</b>",
                f"👷 Data producer : <b>{format_number(Dataproducer)}</b>",
                f"❌ Not ICP : <b>{format_number(Noticp)}</b>",
            ]
        ),
        "button_1": emailbuilder.button(
            link=(f"https://naas.ai/chat/use?plugin_url={plugin}"),
            text="Start Chatting With Agent",
            background_color="#181a1c",
        ),
        "footer_cs": emailbuilder.footer_company(naas=True),
    }

    email_content = emailbuilder.generate(display="iframe", **content)
    return email_content


email_content = email_brief(
    today,
    Dataconsummer,
    Dataproducer,
    Noticp,
)

## Output

### Send email notification

In [ ]:
# sends the email
naas.notification.send(
    email_to=EMAIL_TO, subject=EMAIL_SUBJECT, html=email_content, email_from=EMAIL_FROM
)